# A proper way to run things periodically

One of our core requirements is to be able to call things periodically. 
To allow good accuracy in the timing, special care has to be taken.
Let's have a closer look ...

In [1]:
import time
import random
import asyncio

import app_state  # for singleton 'keep_running'

In [2]:
# override print to allow showing the output in one cell as it was printed in a terminal
import os

real_print = print
lines = []
def print(text):
    global lines
    lines.append(text)

In [3]:
exit_after = 5    # seconds
call_every = 0.5  # seconds
print(f"Work will be done after {exit_after:.1f} seconds.")

In [4]:
real_print("\n".join(lines))
lines = []

Work will be done after 5.0 seconds.


Here's an example of a blocking work load that should be called periodically.

In [5]:
def do_work():
    work_for = call_every * random.random()
    time.sleep(work_for)
    print(f"Done with work item @{time.perf_counter() - start:.1f} seconds.")

And this function helps us terminating everything when the time has come.

In [10]:
async def exit_after(exit_after):
    
    await asyncio.sleep(exit_after)
    app_state.keep_running = False

The actual asynchronouse implementation simply wraps the calls in an endless `while` loop.
Execution time of the callback is measured to modify the final sleep time.
By doing so the timing can be kept.
The function terminates if the singleton `app_state.keep_running` has been set to `False` somewhere.

In [11]:
async def periodically(call_every, callback):
    while app_state.keep_running:
        
        tic  = time.perf_counter()
        callback()
        toc = time.perf_counter()
    
        sleep_for = max(0, call_every - (toc - tic))
        await asyncio.sleep(sleep_for)

And this is how it looks like ...

In [12]:
app_state.keep_running = True

start = time.perf_counter()
_ = await asyncio.gather(
    exit_after(5),
    periodically(call_every, lambda : do_work())
)
end = time.perf_counter()
print(f"All work completed after {end - start:.1f} seconds.")

In [13]:
real_print("\n".join(lines))
lines = []

Done with work item @0.1 seconds.
Done with work item @0.6 seconds.
Done with work item @1.1 seconds.
Done with work item @1.7 seconds.
Done with work item @2.1 seconds.
Done with work item @3.0 seconds.
Done with work item @3.0 seconds.
Done with work item @3.6 seconds.
Done with work item @4.4 seconds.
Done with work item @4.9 seconds.
All work completed after 5.0 seconds.


That's it!